#### Pipeline ETL para ingesta en base de datos relacional

In [1]:
# importamos las librerias necesarias
import pandas as pd
import json
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import gzip

In [2]:
# Ruta al archivo Parquet
ruta_archivo = ".\metadata.parquet"

# Leer el archivo Parquet en un DataFrame de Pandas
df_metadata = pd.read_parquet(ruta_archivo)



In [3]:
# Eliminamos columnas irrelevantes
df_metadata.drop(columns=['address', 'state', 'relative_results', 'url','hours','description', 'price'], inplace=True)

In [4]:
from pandas import json_normalize 

# Desanidar los datos de la columna "MISC"
df_desanidado = json_normalize(df_metadata['MISC'])

In [5]:
# Vamos a trabajar con una copia de metadata para crear una posible tabla "miselaneos"
gmap_id = df_metadata.copy()

# Resetear el índice del DataFrame df_gmap_id
gmap_id.reset_index(drop=True, inplace=True)

# Combinar los DataFrames desanidados
miselaneos = pd.concat([gmap_id, df_desanidado], axis=1)

In [6]:
# Eliminamos columna MISC
df_metadata.drop(columns=['MISC'], inplace=True)

In [7]:
# Eliminamos columna MISC
df_metadata.drop(columns=['category'], inplace=True)

In [ ]:
# DEBERIAMOS QUEDARNOS UNICAMENTE CON EL PRIMER VALOR DE LA COLUMNA CATEGORY

In [8]:
# Verificar si hay valores duplicados
df_metadata.duplicated().value_counts()

False    2998428
True       26583
Name: count, dtype: int64

In [9]:
# Eliminamos duplicados
df_metadata.drop_duplicates(inplace=True)

In [10]:
# Verificar si hay valores nulos
df_metadata.isnull().sum()

name              37
gmap_id            0
latitude           0
longitude          0
avg_rating         0
num_of_reviews     0
dtype: int64

In [22]:
df_metadata.shape

(2998428, 6)

In [11]:
# cambio el nombre de la columna gmap_id a business_id 
df_metadata.rename(columns={'gmap_id': 'business_id'}, inplace=True)

In [14]:
import geopandas as gpd
from shapely.geometry import Point

# Cargar los datos de los estados de Estados Unidos desde Natural Earth
us_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Función para obtener el estado dado una latitud y longitud
def obtener_estado(lat, lon):
    point = Point(lon, lat)
    for index, state in us_states.iterrows():
        if state['geometry'].contains(point):
            return state['name']
    return None

# Aplicar la función a cada fila del DataFrame y crear una nueva columna 'state'
df_metadata['state'] = df_metadata.apply(lambda row: obtener_estado(row['latitude'], row['longitude']), axis=1)


C:\Users\sebas\AppData\Local\Temp\ipykernel_5156\2253121525.py:5: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  us_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [ ]:
# Verificamos tamaño de dataset
df_metadata.shape

(0, 7)

In [82]:
# Observamos resultados
df_metadata.head(5)

,name,gmap_id,latitude,longitude,avg_rating,num_of_reviews
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,4.9,16
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.215290,4.5,6
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,4.4,18
3,Nova Fabrics,0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.232930,3.3,6
4,Nobel Textile Co,0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,4.3,7


In [ ]:
"""

# Exportamos el dataset
df_metadata.to_csv('metadata.csv', index=False)
df_metadata.to_parquet('metadata.parquet')

"""